In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create a SparkSession
spark = SparkSession.builder.appName("PatentTopNAnalytics") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()


24/11/25 18:23:19 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.33 instead (on interface en0)
24/11/25 18:23:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 18:23:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
branches_data = [
    
    ("Artificial Intelligence", ["G06N", "G06F19/00", "G06F17/27", "G06F17/28", "G10L15/22", "G06T", "G06K9/00"]),
    ("Data Science and Analytics", ["G06F16/00", "G06F17/00", "H04L67/00", "H04L12/28", 
                                    "G06F3/06", "G06F12/02", "G06F16/27", "H04L29/06", "G06Q50/00", "B82Y30/00"]),
    ("Networking and Distributed Systems", ["H04L29/00", "H04L12/00", "H04W", "H04L12/24", "H04L67/10", "G06F9/50", "H04L67/22", "H04L29/08", "G06F9/46", "G06F15/173", "G06F15/16"]),
    ("Software Development and Security", ["G06F8/00", "G06F9/00", "G06F9/44", "G06F9/451", "H04L67/02", "G06F3/048", "G06Q10/00","G06F3/00", "H04L9/00", "G06F21/00", "G06Q20/40"]),
    ("Advanced Computing Technologies", ["B25J9/00", "G05B19/00", "G06F15/18", "H03K17/00", "G09G5/00", "G06Q30/00"])
       
]

# Create the branches DataFrame
branches_df = spark.createDataFrame(branches_data, ["branch", "codes"])

branches_df_exploded = branches_df.withColumn("code", explode(col("codes"))).drop("codes")

branches_df_exploded.printSchema()
branches_df_exploded.show(truncate=False)

root
 |-- branch: string (nullable = true)
 |-- code: string (nullable = true)



+----------------------------------+---------+
|branch                            |code     |
+----------------------------------+---------+
|Artificial Intelligence           |G06N     |
|Artificial Intelligence           |G06F19/00|
|Artificial Intelligence           |G06F17/27|
|Artificial Intelligence           |G06F17/28|
|Artificial Intelligence           |G10L15/22|
|Artificial Intelligence           |G06T     |
|Artificial Intelligence           |G06K9/00 |
|Data Science and Analytics        |G06F16/00|
|Data Science and Analytics        |G06F17/00|
|Data Science and Analytics        |H04L67/00|
|Data Science and Analytics        |H04L12/28|
|Data Science and Analytics        |G06F3/06 |
|Data Science and Analytics        |G06F12/02|
|Data Science and Analytics        |G06F16/27|
|Data Science and Analytics        |H04L29/06|
|Data Science and Analytics        |G06Q50/00|
|Data Science and Analytics        |B82Y30/00|
|Networking and Distributed Systems|H04L29/00|
|Networking a

In [3]:
cpc_df = spark.read.parquet("input_parquet/cpc_info")  

# Join condition: exact match for cpc_group and regex match for cpc_subclass
filtered_patents = cpc_df.join(
    branches_df_exploded,
    (cpc_df.cpc_group == branches_df_exploded.code) |  # Exact match for group
    (cpc_df.cpc_subclass == branches_df_exploded.code),
    "inner"
).select("patent_id", "branch", "code")

# Show filtered patents
filtered_patents.show(truncate=False)

filtered_patents.printSchema()

+---------+-----------------------+----+
|patent_id|branch                 |code|
+---------+-----------------------+----+
|4575628  |Artificial Intelligence|G06T|
|4575628  |Artificial Intelligence|G06T|
|4575628  |Artificial Intelligence|G06T|
|4575751  |Artificial Intelligence|G06T|
|4349739  |Artificial Intelligence|G06T|
|4349739  |Artificial Intelligence|G06T|
|4349739  |Artificial Intelligence|G06T|
|4533957  |Artificial Intelligence|G06T|
|4533959  |Artificial Intelligence|G06T|
|4670781  |Artificial Intelligence|G06T|
|4670788  |Artificial Intelligence|G06T|
|4670788  |Artificial Intelligence|G06T|
|4670788  |Artificial Intelligence|G06T|
|4670788  |Artificial Intelligence|G06T|
|4670793  |Artificial Intelligence|G06T|
|4670793  |Artificial Intelligence|G06T|
|4486784  |Artificial Intelligence|G06T|
|4670840  |Artificial Intelligence|G06T|
|4670840  |Artificial Intelligence|G06T|
|4670848  |Artificial Intelligence|G06N|
+---------+-----------------------+----+
only showing top

In [4]:
unique_patents_df = filtered_patents.dropDuplicates(["patent_id"])

unique_patents_df.show(truncate=False)

#record_count = filtered_patents.count()
#print(f"Total number of records: {record_count}")

#record_count = unique_patents_df.count()
#print(f"Total number of unique records: {record_count}")

+---------+----------------------------------+---------+
|patent_id|branch                            |code     |
+---------+----------------------------------+---------+
|10000036 |Data Science and Analytics        |B82Y30/00|
|10001293 |Networking and Distributed Systems|H04L67/10|
|10001380 |Networking and Distributed Systems|H04W     |
|10001910 |Software Development and Security |G06F3/048|
|10002005 |Software Development and Security |G06F9/451|
|10002009 |Software Development and Security |G06F21/00|
|10002036 |Networking and Distributed Systems|H04L67/10|
|10002095 |Artificial Intelligence           |G06N     |
|10002107 |Artificial Intelligence           |G06N     |
|10002157 |Data Science and Analytics        |G06F16/27|
|10002302 |Networking and Distributed Systems|H04W     |
|10002323 |Artificial Intelligence           |G06N     |
|10002325 |Artificial Intelligence           |G06N     |
|10002327 |Artificial Intelligence           |G06N     |
|10002337 |Artificial Intellige

In [5]:
patent_df = spark.read.parquet("input_parquet/patent_info")

patents_filtered_df = patent_df.join(
    unique_patents_df,
    patent_df.patent_id == unique_patents_df.patent_id, 
    "inner"
)

patents_filtered_df = patents_filtered_df.drop(patent_df['patent_id'])
patents_filtered_df.show(truncate=False)

+-----------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------+-----------+-----------+---------+----------------------------------+---------+
|patent_type|patent_date|patent_title                                                                                                                                           |num_claims|application_id|filing_date|series_code|patent_id|branch                            |code     |
+-----------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------+-----------+-----------+---------+----------------------------------+---------+
|utility    |2018-06-19 |High kinetic energy penetrator shielding and high wear resistance materials fabricated with boron nitride nanotubes (BNNTS) an

In [6]:
inventor_data_path = "input_parquet/inventor_info"  
inventor_df = spark.read.parquet(inventor_data_path)

inventor_filtered_df = inventor_df.join(
    patents_filtered_df,
    inventor_df.patent_id == patents_filtered_df.patent_id, 
    "inner"
)

inventor_filtered_df = inventor_filtered_df.drop(inventor_df['patent_id'])

In [7]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

grouped_df = inventor_filtered_df.groupBy(
    "branch", 
    "inventor_id", 
    "disambig_inventor_name_first", 
    "disambig_inventor_name_last", 
    "gender_code", 
    "latitude",
    "longitude",
    "disambig_state"
).agg(
    F.count("patent_id").alias("patent_count")  # Count the number of patents per inventor per branch
)

# Step 2: Define a window specification to rank inventors within each branch
window_spec = Window.partitionBy("branch").orderBy(F.desc("patent_count"))

# Step 3: Add rank column based on patent_count
ranked_df = grouped_df.withColumn("rank", F.row_number().over(window_spec))

# Step 4: Filter top 20 inventors in each branch
top_inventors = ranked_df.filter(F.col("rank") <= 20)

# Step 5: Rename fields and select the desired columns
#F.concat(F.col("disambig_inventor_name_first"), F.lit(" "), F.col("disambig_inventor_name_last")).alias("inventor_name"),
final_result = top_inventors.select(
    F.col("branch"),
    F.col("inventor_id"),
    F.concat(F.col("disambig_inventor_name_first"), F.lit(" "), F.col("disambig_inventor_name_last")).alias("inventor_name"),
    F.col("gender_code").alias("gender"),
    F.col("disambig_state").alias("state"),
    F.col("latitude"),
    F.col("longitude"),
    F.col("patent_count"),
    F.col("rank")
).orderBy("branch", "rank")

final_result.show(truncate=False)



+-------------------------------+----------------------+--------------------+------+-----+----------+------------+------------+----+
|branch                         |inventor_id           |inventor_name       |gender|state|latitude  |longitude   |patent_count|rank|
+-------------------------------+----------------------+--------------------+------+-----+----------+------------+------------+----+
|Advanced Computing Technologies|fl:ja_ln:walker-54    |Jay S. Walker       |M     |CT   |41.2814842|-73.4981792 |34          |1   |
|Advanced Computing Technologies|fl:ma_ln:stefik-1     |Mark J. Stefik      |M     |CA   |37.3736298|-122.219047 |26          |2   |
|Advanced Computing Technologies|fl:pe_ln:pirolli-1    |Peter L. T. Pirolli |M     |CA   |37.7790262|-122.419906 |24          |3   |
|Advanced Computing Technologies|fl:sh_ln:hussain-12   |Shariq Hussain      |M     |CA   |33.2000368|-117.2425355|20          |4   |
|Advanced Computing Technologies|fl:ia_ln:miller-9     |Ian Miller   

In [8]:
final_result_with_schema = final_result.select(
    col("branch").cast(StringType()),
    col("inventor_id").cast(StringType()),
    col("inventor_name").cast(StringType()),
    col("gender").cast(StringType()),
    col("state").cast(StringType()),
    col("latitude").cast(DoubleType()),
    col("longitude").cast(DoubleType()),
    col("patent_count").cast(IntegerType()),
    col("rank").cast(IntegerType())
)

final_result_with_schema.printSchema()
# Show the final result

root
 |-- branch: string (nullable = true)
 |-- inventor_id: string (nullable = true)
 |-- inventor_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- patent_count: integer (nullable = false)
 |-- rank: integer (nullable = false)



In [9]:
final_result_with_schema.write.parquet("analytics_output_parquet/top_inventors", mode="overwrite")

In [10]:
geo_data = inventor_filtered_df.groupBy("branch", "disambig_state").agg(
    F.avg("latitude").alias("latitude"),  
    F.avg("longitude").alias("longitude"), 
    F.countDistinct("inventor_id").alias("inventor_count"),  # Count unique inventors
    F.countDistinct("patent_id").alias("patent_count")  # Count unique patents
)

# Step 2: Select relevant columns
geo_distribution = geo_data.select(
    F.col("branch"),
    F.concat(F.lit("US-"), F.col("disambig_state")).alias("state"),
    F.col("latitude"),
    F.col("longitude"),
    F.col("inventor_count"),
    F.col("patent_count")
)


# Show the resulting DataFrame
geo_distribution.show(truncate=False)

geo_distribution_with_schema = geo_distribution.select(
    col("branch").cast(StringType()),
    col("state").cast(StringType()),
    col("latitude").cast(DoubleType()),
    col("longitude").cast(DoubleType()),
    col("inventor_count").cast(IntegerType()),
    col("patent_count").cast(IntegerType())
)

geo_distribution_with_schema.printSchema()

+----------------------------------+-----+------------------+-------------------+--------------+------------+
|branch                            |state|latitude          |longitude          |inventor_count|patent_count|
+----------------------------------+-----+------------------+-------------------+--------------+------------+
|Artificial Intelligence           |US-WY|41.902003858823534|-106.46323838352941|45            |70          |
|Software Development and Security |US-FL|27.280714325476314|-81.08960283375069 |806           |1108        |
|Data Science and Analytics        |US-MD|39.147088231774056|-76.87314561800527 |491           |592         |
|Advanced Computing Technologies   |US-WV|38.8935492        |-79.14055222       |3             |5           |
|Networking and Distributed Systems|US-PA|40.27709383376434 |-76.07247477966796 |1784          |4348        |
|Software Development and Security |US-CO|39.721263518850876|-105.11172825712248|890           |1038        |
|Networkin

In [11]:
geo_distribution_with_schema.write.parquet("analytics_output_parquet/geo_distribution", mode="overwrite")

In [12]:
applicant_data_path = "input_parquet/applicant_info"  # Path to the patent data
applicant_df = spark.read.parquet(applicant_data_path)

applicant_filtered_df = applicant_df.join(
    patents_filtered_df,
    applicant_df.patent_id == patents_filtered_df.patent_id, 
    "inner"
)

applicant_filtered_df = applicant_filtered_df.drop(applicant_df['patent_id'])
applicant_filtered_df.printSchema()

root
 |-- applicant_sequence: string (nullable = true)
 |-- raw_applicant_name_first: string (nullable = true)
 |-- raw_applicant_name_last: string (nullable = true)
 |-- raw_applicant_organization: string (nullable = true)
 |-- applicant_type: string (nullable = true)
 |-- patent_type: string (nullable = true)
 |-- patent_date: string (nullable = true)
 |-- patent_title: string (nullable = true)
 |-- num_claims: string (nullable = true)
 |-- application_id: string (nullable = true)
 |-- filing_date: string (nullable = true)
 |-- series_code: string (nullable = true)
 |-- patent_id: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- code: string (nullable = true)



In [20]:
from pyspark.sql import Window
from pyspark.sql import functions as F

# Step 1: Remove NULL values
applicant_cleaned_df = applicant_filtered_df.filter(F.col("raw_applicant_organization").isNotNull())

# Step 2: Normalize organization names to sentence case for deduplication
# This ensures records like "SAMSUNG ELECTRONICS CO., LTD." and "Samsung Electronics Co., Ltd." are treated the same
applicant_normalized_df = applicant_cleaned_df.withColumn(
    "normalized_organization", 
    F.initcap(F.lower(F.col("raw_applicant_organization")))
)

# Step 3: Group by normalized organization and branch, then sum the patent counts
applicant_grouped_df = applicant_normalized_df.groupBy(
    "branch", 
    "normalized_organization"
).agg(
    F.count("patent_id").alias("patent_count")  # Count the number of patents per inventor per branch
)

# Step 4: Define a window specification to rank organizations within each branch
applicant_window_spec = Window.partitionBy("branch").orderBy(F.desc("patent_count"))

# Step 5: Add rank column based on patent_count
applicant_ranked_df = applicant_grouped_df.withColumn(
    "rank", 
    F.row_number().over(applicant_window_spec)
)

# Step 6: Filter top 20 organizations in each branch
top_applicants = applicant_ranked_df.filter(F.col("rank") <= 20)

# Step 7: Rename fields and select the desired columns
final_applicant_result = top_applicants.select(
    F.col("branch"),
    F.col("normalized_organization").alias("organization"),
    F.col("patent_count"),
    F.col("rank")
).orderBy("branch", "rank")

# Step 8: Show the results
final_applicant_result.show(truncate=False)

+-------------------------------+----------------------------------------------------+------------+----+
|branch                         |organization                                        |patent_count|rank|
+-------------------------------+----------------------------------------------------+------------+----+
|Advanced Computing Technologies|Samsung Electronics Co., Ltd.                       |110         |1   |
|Advanced Computing Technologies|Sony Corporation                                    |66          |2   |
|Advanced Computing Technologies|Amazon Technologies, Inc.                           |48          |3   |
|Advanced Computing Technologies|Google Inc.                                         |43          |4   |
|Advanced Computing Technologies|International Business Machines Corporation         |41          |5   |
|Advanced Computing Technologies|Samsung Display Co., Ltd.                           |40          |6   |
|Advanced Computing Technologies|Ebay Inc.             

In [21]:
final_applicant_with_schema = final_applicant_result.select(
    col("branch").cast(StringType()),
    col("organization").cast(StringType()),
    col("patent_count").cast(IntegerType()),
    col("rank").cast(IntegerType())
)

In [22]:
final_applicant_with_schema.write.parquet("analytics_output_parquet/top_applicants", mode="overwrite")